# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 19 2022 11:48AM,237648,10,8039814,Beach Products Inc,Released
1,May 19 2022 11:48AM,237648,10,8039817,Beach Products Inc,Released
2,May 19 2022 11:48AM,237648,10,8039818,Beach Products Inc,Released
3,May 19 2022 11:48AM,237648,10,8039819,Beach Products Inc,Released
4,May 19 2022 11:48AM,237648,10,8039820,Beach Products Inc,Released
5,May 19 2022 11:48AM,237648,10,8039821,Beach Products Inc,Released
6,May 19 2022 11:48AM,237648,10,8039822,Beach Products Inc,Released
7,May 19 2022 11:48AM,237648,10,8039823,Beach Products Inc,Released
8,May 19 2022 11:48AM,237648,10,8039824,Beach Products Inc,Released
9,May 19 2022 11:48AM,237648,10,8039825,Beach Products Inc,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,237644,Released,20
43,237645,Released,4
44,237646,Released,6
45,237647,Released,2
46,237648,Released,13


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237644,NaN,NaN,20.0
237645,NaN,NaN,4.0
237646,NaN,NaN,6.0
237647,NaN,NaN,2.0
237648,NaN,NaN,13.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237550,0.0,0.0,1.0
237557,0.0,21.0,7.0
237566,0.0,0.0,4.0
237567,0.0,0.0,1.0
237571,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237550,0,0,1
237557,0,21,7
237566,0,0,4
237567,0,0,1
237571,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237550,0,0,1
1,237557,0,21,7
2,237566,0,0,4
3,237567,0,0,1
4,237571,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237550,,,1
1,237557,,21,7
2,237566,,,4
3,237567,,,1
4,237571,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 19 2022 11:48AM,237648,10,Beach Products Inc
13,May 19 2022 11:46AM,237647,10,Emerginnova
15,May 19 2022 11:38AM,237646,18,Eye Pharma Inc
21,May 19 2022 11:34AM,237645,10,"Methapharm, Inc."
25,May 19 2022 11:34AM,237644,10,"Methapharm, Inc."
45,May 19 2022 11:32AM,237642,18,Hush Hush
46,May 19 2022 11:32AM,237643,18,Hush Hush
47,May 19 2022 11:08AM,237641,22,"NBTY Global, Inc."
48,May 19 2022 11:06AM,237640,22,"NBTY Global, Inc."
49,May 19 2022 11:04AM,237639,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 19 2022 11:48AM,237648,10,Beach Products Inc,,,13
1,May 19 2022 11:46AM,237647,10,Emerginnova,,,2
2,May 19 2022 11:38AM,237646,18,Eye Pharma Inc,,,6
3,May 19 2022 11:34AM,237645,10,"Methapharm, Inc.",,,4
4,May 19 2022 11:34AM,237644,10,"Methapharm, Inc.",,,20
5,May 19 2022 11:32AM,237642,18,Hush Hush,,,1
6,May 19 2022 11:32AM,237643,18,Hush Hush,,,1
7,May 19 2022 11:08AM,237641,22,"NBTY Global, Inc.",,,1
8,May 19 2022 11:06AM,237640,22,"NBTY Global, Inc.",,,1
9,May 19 2022 11:04AM,237639,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 11:48AM,237648,10,Beach Products Inc,13,,
1,May 19 2022 11:46AM,237647,10,Emerginnova,2,,
2,May 19 2022 11:38AM,237646,18,Eye Pharma Inc,6,,
3,May 19 2022 11:34AM,237645,10,"Methapharm, Inc.",4,,
4,May 19 2022 11:34AM,237644,10,"Methapharm, Inc.",20,,
5,May 19 2022 11:32AM,237642,18,Hush Hush,1,,
6,May 19 2022 11:32AM,237643,18,Hush Hush,1,,
7,May 19 2022 11:08AM,237641,22,"NBTY Global, Inc.",1,,
8,May 19 2022 11:06AM,237640,22,"NBTY Global, Inc.",1,,
9,May 19 2022 11:04AM,237639,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 11:48AM,237648,10,Beach Products Inc,13,,
1,May 19 2022 11:46AM,237647,10,Emerginnova,2,,
2,May 19 2022 11:38AM,237646,18,Eye Pharma Inc,6,,
3,May 19 2022 11:34AM,237645,10,"Methapharm, Inc.",4,,
4,May 19 2022 11:34AM,237644,10,"Methapharm, Inc.",20,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 11:48AM,237648,10,Beach Products Inc,13.0,NaN,NaN
1,May 19 2022 11:46AM,237647,10,Emerginnova,2.0,NaN,NaN
2,May 19 2022 11:38AM,237646,18,Eye Pharma Inc,6.0,NaN,NaN
3,May 19 2022 11:34AM,237645,10,"Methapharm, Inc.",4.0,NaN,NaN
4,May 19 2022 11:34AM,237644,10,"Methapharm, Inc.",20.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 11:48AM,237648,10,Beach Products Inc,13.0,0.0,0.0
1,May 19 2022 11:46AM,237647,10,Emerginnova,2.0,0.0,0.0
2,May 19 2022 11:38AM,237646,18,Eye Pharma Inc,6.0,0.0,0.0
3,May 19 2022 11:34AM,237645,10,"Methapharm, Inc.",4.0,0.0,0.0
4,May 19 2022 11:34AM,237644,10,"Methapharm, Inc.",20.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3326815,212.0,13.0,0.0
15,712869,13.0,6.0,0.0
16,475203,2.0,0.0,0.0
18,2376080,18.0,0.0,0.0
19,712875,6.0,23.0,16.0
20,950276,55.0,21.0,0.0
21,237575,0.0,1.0,0.0
22,950491,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3326815,212.0,13.0,0.0
1,15,712869,13.0,6.0,0.0
2,16,475203,2.0,0.0,0.0
3,18,2376080,18.0,0.0,0.0
4,19,712875,6.0,23.0,16.0
5,20,950276,55.0,21.0,0.0
6,21,237575,0.0,1.0,0.0
7,22,950491,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,212.0,13.0,0.0
1,15,13.0,6.0,0.0
2,16,2.0,0.0,0.0
3,18,18.0,0.0,0.0
4,19,6.0,23.0,16.0
5,20,55.0,21.0,0.0
6,21,0.0,1.0,0.0
7,22,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,212.0
1,15,Released,13.0
2,16,Released,2.0
3,18,Released,18.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0
Executing,13.0,6.0,0.0,0.0,23.0,21.0,1.0,1.0
Released,212.0,13.0,2.0,18.0,6.0,55.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0
1,Executing,13.0,6.0,0.0,0.0,23.0,21.0,1.0,1.0
2,Released,212.0,13.0,2.0,18.0,6.0,55.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0
1,Executing,13.0,6.0,0.0,0.0,23.0,21.0,1.0,1.0
2,Released,212.0,13.0,2.0,18.0,6.0,55.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()